# README

This readme file contains documentation for combining and processing a dataset related to Wikipedia page traffic and analyzing it.


Below are the steps involved
## Gather the data
We need to collect monthly data from the below
There are 5 things here
Legacy Pagecounts API
1. Desktop data
2. Mobile data

Pageviews API
1. Desktop data
2. Mobile web data
3. Mobile app traffic data

### Imports
Before we do any processing, we need to import all required libraries for python. These are available on PyPi repository

In [112]:
import json
import requests
import pandas as pd
import datetime

### Constants and common definitions
The below code will declare constants like endpoints, headers and define the access points supported by the different endpoints

In [70]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/en.wikipedia.org/{access}/monthly/{start}/{end}'
endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/en.wikipedia.org/{access}/{agent}/monthly/{start}/{end}'
headers = {
    'User-Agent': 'https://github.com/vikrantb',
    'From': 'vikrantb@uw.edu'
}

legacy_access_points = ['desktop-site', 'mobile-site']
page_view_access_points = ['desktop', 'mobile-app', 'mobile-web']

### Common helper functions

Below are some helper functions that will be used throughout. With an explanation

#### api_call
This method makes a call to the endpoint provided with the parameters provided. It also has an option to directly extract the items

#### get_wiki_df
This method returns the pagecount/pageviews information in a dataframe format

#### get_from_legacy
This method is a wrapper method on top of api_call specifically for legacy endpoint

#### get_from_page_views
This method is a wrapper method on top of api_call specifically for pageviews endpoint

In [92]:
def api_call(endpoint,parameters, extract_items=False):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response['items'] if extract_items else response

def get_wiki_df(file_path):
    wiki_json = ''
    with open(file_path) as f:
        wiki_json = str(json.load(f)['items']).replace("\'", "\"")
    return pd.read_json(wiki_json, orient='values', convert_dates=False)


def get_from_legacy(access, start, end, save_file=False):
    params = {"project" : "en.wikipedia.org",
             "access" : access,
             "granularity" : "monthly",
             "start" : "{start}0100".format(start=start),
             "end" : "{end}0100".format(end=end)
            }
    data = api_call(endpoint=endpoint_legacy, parameters=params)
    file_name="pagecounts_{access}_{start}-{end}.json".format(access=access, start=start, end=end)
    with open(file_name, 'w') as f:
        json.dump(data, f)


def get_from_page_views(access, start, end, save_file=False):
    params = {"project" : "en.wikipedia.org",
             "access" : access,
             "agent" : "user",
             "granularity" : "monthly",
             "start" : "{start}0100".format(start=start),
             "end" : "{end}0100".format(end=end)
            }
    data = api_call(endpoint=endpoint_pageviews, parameters=params)
    file_name="pageviews_{access}_{start}-{end}.json".format(access=access, start=start, end=end)
    with open(file_name, 'w') as f:
        json.dump(data, f)

## Step 1: Gathering the data

In this step we will gather the wikipedia pagecount data per month level.
As noted below, the data is available in legacy endpoint from January 2008 to July 2016
As noted below, the data is available in pageviews endpoint from July 2015 2008 to September 2020.

There is an overlap for data from July 2015 to July 2016

After execution of the below step, there will be 5 json files created as below

- pagecounts_desktop-site_200801-201607.json - desktop site information from legacy api from January 2008 to July 2016
- pagecounts_mobile-site_200801-201607.json - mobile site information from legacy api from January 2008 to July 2016
- pageviews_desktop_201507-202009.json - desktop information from pageviews api from July 2015 to September 2020
- pageviews_mobile-app_201507-202009.json - mobile app information from pageviews api from July 2015 to September 2020
- pageviews_mobile-web_201507-202009.json - mobile web information from pageviews api from July 2015 to September 2020

In [74]:
legacy_start_date='200801'
legacy_end_date='201607'
for access_type in legacy_access_points:
    get_from_legacy(access=access_type, start=legacy_start_date,end=legacy_end_date, save_file=True)

legacy_start_date='201507'
legacy_end_date='202009'
for access_type in page_view_access_points:
    get_from_page_views(access=access_type, start=legacy_start_date,end=legacy_end_date, save_file=True)

## Step 2: Processing the data
The objective of this step is to combine data from all the json files collected in the gathering step into a single csv file upon which we will run analysis.

For the pageviews api, we will sum the counts from mobile-app and mobile-web to create a single value for mobile traffic.

For all the data, we will be only using the `YYYYMM` part of the timestamp and ignoring the `DDHH` part

In [109]:
pagecounts_desktop_fp = 'pagecounts_desktop-site_200801-201607.json'
pagecounts_mobile_fp = 'pagecounts_mobile-site_200801-201607.json'
pageviews_desktop_fp = 'pageviews_desktop_201507-202009.json'
pageviews_mobile_app_fp = 'pageviews_mobile-app_201507-202009.json'
pageviews_mobile_web_fp = 'pageviews_mobile-web_201507-202009.json'

pagecounts_desktop_df = get_wiki_df(pagecounts_desktop_fp)
pagecounts_mobile_df = get_wiki_df(pagecounts_mobile_fp)
pageviews_desktop_df = get_wiki_df(pageviews_desktop_fp)
pageviews_mobile_app_df = get_wiki_df(pageviews_mobile_app_fp)
pageviews_mobile_web_df = get_wiki_df(pageviews_mobile_web_fp)




In [111]:
pageviews_mobile_app_df['timestamp'] = pageviews_mobile_app_df['timestamp'].apply(str)
pageviews_mobile_app_df['timestamp'] = pageviews_mobile_app_df.apply(lambda row: row['timestamp'][:6], axis=1)
pageviews_mobile_app_df.head(2)
datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')

,project,access,agent,granularity,timestamp,views
0,en.wikipedia,mobile-app,user,monthly,201507,109624146
1,en.wikipedia,mobile-app,user,monthly,201508,109669149


In [102]:
pageviews_mobile_app_df.dtypes

project        object
access         object
agent          object
granularity    object
timestamp       int64
views           int64
dtype: object